<font size = "5">

Explanation of mean reversion trading strategy

In [66]:
import pandas as pd
import numpy as np

<font size = "4">

Step 0: Collect our data from our priceData function. For this walkthrough we will use AAPL as an example.

In [67]:
from quantlab.data.yf import priceData
from quantlab.data.tickers import tickers

data = priceData(tickers, 150)
s = pd.Series(data["AAPL"])

/Users/ryanvandeberghe/Documents/GitHub/quantlab/quantlab/data/yf.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
[*********************100%***********************]  239 of 239 completed

32 Failed downloads:
['ALL', 'ZTS', 'AMZN', 'BMY', 'BIO', 'ZION', 'CSCO', 'STLA', 'STT', 'ADBE', 'CB', 'HMC', 'BK', 'KRC', 'ROKU', 'VRTX', 'COIN', 'PSX', 'UNH', 'SNOW', 'FITB', 'CAH', 'XOM', 'ESS', 'BA', 'SBUX', 'KKR']: Timeout('Failed to perform, curl: (28) Connection timed out after 10002 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['OXY', 'FIS']: Timeout('Failed to perform, curl: (28) Connection timed out after 10003 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['MRK', 'A', 'IVZ']: Timeout('Failed to perform, curl: (28) Connection timed out after 10001 milliseconds. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


<font size = "4">

Step 1: For each stock, create arrays of the stock orderings ``t``, prices ``s``, and price logarithms ``y``. We use log to compare changes more accurately.

In [68]:
n = len(s)
t = np.arange(n)
y = np.log(s.values)

In [69]:
print(f"n: {n}")
print(f"t: {t[:10]} ...")
print(f"y: {y[:10]} ...")

n: 104
t: [0 1 2 3 4 5 6 7 8 9] ...
y: [5.30564716 5.31340087 5.31118467 5.30032821 5.31659347 5.30440696
 5.31044476 5.29106441 5.29317508 5.27927366] ...


<font size = "4">

Step 2: Calculate mean values of ``y`` and ``t`` to build a regression line of the price data over the past ``n`` days. Then, create a series of expected y values ``exp_ys`` and expected prices ``exp_ps`` to compare with our actual y values.

In [70]:
t_mean = t.mean()
y_mean = y.mean()
slope = np.sum((t - t_mean) * (y - y_mean)) / np.sum((t - t_mean)**2)
intercept = y_mean - slope * t_mean
exp_ys = intercept + slope * t
exp_ps = np.exp(exp_ys)

In [71]:
print(f"t_mean: {t_mean}")
print(f"y_mean: {y_mean}")
print(f"slope: {slope}")
print(f"intercept: {intercept}")
print(f"exp_ys: {exp_ys[:10]} ...")
print(f"exp_ps: {exp_ps[:10]} ...")

t_mean: 51.5
y_mean: 5.419762947034555
slope: 0.002983531006918908
intercept: 5.2661111001782315
exp_ys: [5.2661111  5.26909463 5.27207816 5.27506169 5.27804522 5.28102876
 5.28401229 5.28699582 5.28997935 5.29296288] ...
exp_ps: [193.66136647 194.24002395 194.82041046 195.40253115 195.98639121
 196.57199583 197.15935024 197.74845965 198.33932931 198.93196448] ...


<font size = "4">

Step 3: Compare our expectations with the actual prices and calculate ``error_threshold`` based on standard deviation of our errors. Then, compare ``current_price`` with ``expected_current_price`` to figure out the ``error``. If the error is significant enough, we believe the stock is overvalued/undervalued and we should make a move on it.

In [72]:
residuals = s.values - exp_ps
error_threshold = residuals.std()
current_price = s.iloc[-1]
exp_current_price = exp_ps[-1]
error = current_price - exp_current_price

In [73]:
print(f"residuals: {residuals[:10]} ...")
print(f"error_threshold: {error_threshold}")
print(f"current_price: {current_price}")
print(f"exp_current_price: {exp_current_price}")
print(f"error: {error}")

residuals: [ 7.80997753  8.79954209  7.76967743  5.00004148  7.70243631  4.64963808
  5.28089634  0.80619733  0.63485282 -2.70466956] ...
error_threshold: 6.043211934313763
current_price: 269.0
exp_current_price: 263.3317894403118
error: 5.668210559688191


<font size = "4">
Step 4: If the error is over 2 times the error threshold, meaning the stock overprices the mean by 2 standard deviations, we believe that the stock will revert back to its mean and therefore we short the stock now. If the error is less than -2 times the error threshold, the stock underprices the mean by 2 standard deviations, and we long the stock expecting it to grow and revert back to its mean. The amount we buy should be standardized by price, so we divide 100 by the price of a share in order to buy/sell $100 of each stock.

In [74]:
if error > 2 * error_threshold:
    result = -100 / current_price
    print("Since the result is greater than the mean minus 2 standard deviations, short the stock.")
elif error < -2 * error_threshold:
    result = 100 / current_price
    print("Since the result is less than the mean minus 2 standard deviations, long the stock.")
else:
    print("Since the error is within 2 standard deviations of the mean, do nothing.")

Since the error is within 2 standard deviations of the mean, do nothing.


<font size = "4">
Now, Wrap all of this as a function, looping across all columns. Start by initializing an empty data frame ``results``, which will be added to if our result consists of us buying or selling a stock. Return the data frame of trading decisions of all stocks.

In [75]:
def meanReversion(data):
    results = {}
    for col in data.columns:
        s = pd.Series(data[col])
        n = len(s)
        t = np.arange(n)
        y = np.log(s.values)
        t_mean = t.mean()
        y_mean = y.mean()
        slope = np.sum((t - t_mean) * (y - y_mean)) / np.sum((t - t_mean)**2)
        intercept = y_mean - slope * t_mean
        exp_ys = intercept + slope * t
        exp_ps = np.exp(exp_ys)
        residuals = s.values - exp_ps
        error_threshold = residuals.std()
        current_price = s.iloc[-1]
        exp_current_price = exp_ps[-1]
        error = current_price - exp_current_price

        if error > 2 * error_threshold:
            results[col] = -100 / current_price
        elif error < -2 * error_threshold:
            results[col] = 100 / current_price
        else:
            continue
    return pd.Series(results, name = "shares")

In [76]:
results = meanReversion(data)
print(results)

AFG      0.764409
AMD     -0.387582
ARE      1.588815
BLK      0.088893
BX       0.659239
CINF     0.658979
CRWD    -0.182835
D        1.677290
DOW     -3.940110
EW      -1.216693
GM      -1.431229
HIG      0.815062
IBM     -0.319928
ISRG    -0.183355
JNJ      0.534960
KO      -1.425313
LIN      0.225876
MDT      1.072616
MKL      0.054408
MU      -0.450633
NCLH     4.474273
NIO     14.104372
NSC      0.356633
NVS      0.810176
PGR      0.471898
QCOM    -0.552395
REGN    -0.152793
RMD      0.389757
SPG      0.565707
UAL      1.053297
UPS     -1.037775
VICI     3.293808
WPC      1.512630
Name: shares, dtype: float64
